## 0、安装依赖包

In [ ]:
%pip install -U pinecone-client qianfan tiktoken langchain pymupdf -i https://pypi.tuna.tsinghua.edu.cn/simple/

## 1、加载环境变量

新建一个env的文件，然后填写如下内容：<br>
PINECONE_API_KEY="your PINECONE_API_KEY"<br>
QIANFAN_AK="your QIANFAN_AK"<br>
QIANFAN_SK="your QIANFAN_SK"


In [62]:
# 通过配置文件加载环境变量
import os
from dotenv import load_dotenv
load_dotenv(dotenv_path = "env",override = True)

True

## 2、pinecone初始化

初始化pinecone，并创建pinecone索引（index）

In [63]:
import pinecone
from tqdm.autonotebook import tqdm

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

# PINECONE免费用户的默认ENV都是"gcp-starter"
PINECONE_ENV = "gcp-starter"

pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENV
)

index_name = "qianfan-vdb"

if index_name not in pinecone.list_indexes():
    # we create a new index
    pinecone.create_index(
        name=index_name,
        metric='euclidean',
        dimension=384,  # 千帆Embedding-V1向量模型是384维，所以这里设置成384
    )

## 3、千帆SDK初始化鉴权

In [64]:
import os 

QIANFAN_AK=os.getenv("QIANFAN_AK")
QIANFAN_SK=os.getenv("QIANFAN_SK")

os.environ["QIANFAN_AK"] = QIANFAN_AK
os.environ["QIANFAN_SK"] = QIANFAN_SK

## 4、文件切片并保存至向量数据库

In [65]:
from langchain.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("./example_data/ai-paper.pdf")
documents = loader.load()

## 5、切片

初始化vectorstore，使用PyMuPDFLoader加载PDF文件

In [66]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=384, chunk_overlap=0, separators=["\n\n", "\n", " ", "", "。", "，"])
docs_spilts = text_splitter.split_documents(documents)
print(f"{len(docs_spilts)} documents block loaded")

33 documents block loaded


## 6、存储

In [67]:
from langchain.vectorstores import Pinecone
from langchain.embeddings import QianfanEmbeddingsEndpoint

embeddings = QianfanEmbeddingsEndpoint(model="Embedding-V1")
pinecone_vdb = Pinecone.from_documents(docs_spilts, embeddings, index_name=index_name)

## 7、加载已有向量数据库

如果向量数据库已存在，并且已有数据，可以通过Pinecone.from_existing_index加载已有向量数据库

In [68]:
from langchain.vectorstores import Pinecone
from langchain.embeddings import QianfanEmbeddingsEndpoint

index_name = "qianfan-vdb"
embeddings = QianfanEmbeddingsEndpoint(model="Embedding-V1")

Pinecone.from_existing_index(index_name=index_name, embedding=embeddings)

## 8、新建Retriever


In [69]:
retriever = pinecone_vdb.as_retriever(search_type="similarity_score_threshold", search_kwargs={'score_threshold': 0.5, "k":3 })

## 9、查看相似度文档

In [70]:
# 定义query
query = "ERNIE1.0是什么时候发布的？" 

In [71]:
# 检索与query相似度最高的文档
matched_docs = retriever.get_relevant_documents(query)

/Users/xuwei/opt/anaconda3/lib/python3.11/site-packages/langchain_core/vectorstores.py:313: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content='勇在阿里云峰会上正式发布了大语言模型工具通义\n千问，并宣布此语言模型会接入阿里旗下的所有业\n务中。\n百度 ERNIE\n百 度 于 2019 年 3 月 发 布 预 训 练 模 型\nERNIE1.0，2019 年 7 月 发 布 ERNIE2.0，2021\n年 5 月开源四大预训练模型，包括多粒度语言知\n识模型 ERNIE-Gram、超长文本双向建模预训练\n模型 ERNIE-Doc、融合场景图知识的跨模态预训\n练模型 ERNIE-ViL 和语言与视觉一体的预训练\n模型 ERNIE-UNIMO，2021 年 12 月发布多语言\n预训练模型 ERNIE-M。百度持续投入大模型的\n技术创新与产业应用，布局了 NLP、CV、跨模态\n等大模型，率先提出行业大模型，构建大模型工具\n与平台，探索产品与社区，在企业端和用户端均有', metadata={'author': 'CNKI', 'creationDate': "D:20230718144804-08'00'", 'creator': 'ReaderEx_DIS 2.3.0 Build 4031', 'file_path': './example_data/ai-paper.pdf', 'format': 'PDF 1.6', 'keywords': '', 'modDate': '', 'page': 5.0, 'producer': 'TTKN', 'source': './example_data/ai-paper.pdf', 'subject': '', 'title': '', 'total_pages': 7.0, 'trapped': ''}), 1.07225263), (Document(page_content='勇在阿里云峰会上正式发布了大语言模型工具通义\n千问，并宣布此语言模型会接入阿里旗下的所有业\n务中。\n百度 ERNI

## 10、问答

In [72]:
# 新建LLM类对象
from langchain.chat_models import QianfanChatEndpoint

llm = QianfanChatEndpoint(model="ERNIE-Bot-4")

In [73]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

CUSTOM_PROMPT = """
现在你是一个阅读理解机器人，你会阅读并深度理解我给你的文本内容并据此回答我所提出的问题。注意，我给出的问题是：{question} 你需要阅读理解的文本是：{context}
"""

qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever, chain_type_kwargs={"prompt": PromptTemplate.from_template(CUSTOM_PROMPT)}, return_source_documents=True)

In [74]:
qa_chain({"query": query})

/Users/xuwei/opt/anaconda3/lib/python3.11/site-packages/langchain_core/vectorstores.py:313: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content='勇在阿里云峰会上正式发布了大语言模型工具通义\n千问，并宣布此语言模型会接入阿里旗下的所有业\n务中。\n百度 ERNIE\n百 度 于 2019 年 3 月 发 布 预 训 练 模 型\nERNIE1.0，2019 年 7 月 发 布 ERNIE2.0，2021\n年 5 月开源四大预训练模型，包括多粒度语言知\n识模型 ERNIE-Gram、超长文本双向建模预训练\n模型 ERNIE-Doc、融合场景图知识的跨模态预训\n练模型 ERNIE-ViL 和语言与视觉一体的预训练\n模型 ERNIE-UNIMO，2021 年 12 月发布多语言\n预训练模型 ERNIE-M。百度持续投入大模型的\n技术创新与产业应用，布局了 NLP、CV、跨模态\n等大模型，率先提出行业大模型，构建大模型工具\n与平台，探索产品与社区，在企业端和用户端均有', metadata={'author': 'CNKI', 'creationDate': "D:20230718144804-08'00'", 'creator': 'ReaderEx_DIS 2.3.0 Build 4031', 'file_path': './example_data/ai-paper.pdf', 'format': 'PDF 1.6', 'keywords': '', 'modDate': '', 'page': 5.0, 'producer': 'TTKN', 'source': './example_data/ai-paper.pdf', 'subject': '', 'title': '', 'total_pages': 7.0, 'trapped': ''}), 1.07225263), (Document(page_content='勇在阿里云峰会上正式发布了大语言模型工具通义\n千问，并宣布此语言模型会接入阿里旗下的所有业\n务中。\n百度 ERNI

{'query': 'ERNIE1.0是什么时候发布的？',
 'result': '根据提供的文本内容，ERNIE1.0是在2019年3月发布的。\n\n文中提到，百度于2019年3月发布预训练模型ERNIE1.0，2019年7月发布ERNIE2.0。因此，答案是ERNIE1.0在2019年3月发布。',
 'source_documents': [Document(page_content='勇在阿里云峰会上正式发布了大语言模型工具通义\n千问，并宣布此语言模型会接入阿里旗下的所有业\n务中。\n百度 ERNIE\n百 度 于 2019 年 3 月 发 布 预 训 练 模 型\nERNIE1.0，2019 年 7 月 发 布 ERNIE2.0，2021\n年 5 月开源四大预训练模型，包括多粒度语言知\n识模型 ERNIE-Gram、超长文本双向建模预训练\n模型 ERNIE-Doc、融合场景图知识的跨模态预训\n练模型 ERNIE-ViL 和语言与视觉一体的预训练\n模型 ERNIE-UNIMO，2021 年 12 月发布多语言\n预训练模型 ERNIE-M。百度持续投入大模型的\n技术创新与产业应用，布局了 NLP、CV、跨模态\n等大模型，率先提出行业大模型，构建大模型工具\n与平台，探索产品与社区，在企业端和用户端均有', metadata={'author': 'CNKI', 'creationDate': "D:20230718144804-08'00'", 'creator': 'ReaderEx_DIS 2.3.0 Build 4031', 'file_path': './example_data/ai-paper.pdf', 'format': 'PDF 1.6', 'keywords': '', 'modDate': '', 'page': 5.0, 'producer': 'TTKN', 'source': './example_data/ai-paper.pdf', 'subject': '', 'title': '', 'total_pages': 7.0, 'trapped': ''}),
  Document(page_content='勇在阿里云峰会上正式发布了大语言模型工具通义\n千问，并宣布此语言模型会接入阿里旗下的所有业\n